In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data65


In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

Fires  Fires.rar


In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

mkdir: cannot create directory ‘/home/aistudio/external-libraries’: File exists
Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 122kB 19.4MB/s eta 0:00:01


In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

# 配置环境

In [ ]:
# 安装需要的lxml模块
!pip install lxml

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 5.5MB 24.9MB/s eta 0:00:01


In [ ]:
# 安装需要的paddlex模块
!pip install paddlex -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 450kB 13.6MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 13.2MB/s eta 0:00:01
     |████████████████████████████████| 153kB 27.6MB/s eta 0:00:01
     |████████████████████████████████| 348kB 13.2MB/s eta 0:00:01
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=278368 sha256=a837fb2bc3bfedfe2829503a7e21beb1a56abafc8f511469f053b2bad048f753
  Stored in directory: /home/aistudio/.cache/pip/wheels/fb/44/67/8baa69040569b1edbd7776ec6f82c387663e724908aaa60963
Successfully built pycocotools
  Found existing installation: paddlehub 1.6.0
    Uninstalling paddlehub-1.6.0:
      Successfully uninstalled paddlehub-1.6.0


# 数据处理成符合VCO数据集的格式

In [ ]:
# 生成标注XMl
from lxml.etree import Element, SubElement, tostring
from xml.dom.minidom import parseString

def make_xml(xmin_tuple, ymin_tuple, xmax_tuple, ymax_tuple, image_name, width, height):

    node_root = Element('annotation')

    node_folder = SubElement(node_root, 'folder')
    node_folder.text = 'VOC'
    
    # 文件名
    node_filename = SubElement(node_root, 'filename')
    node_filename.text = image_name + '.jpg'

    node_object_num = SubElement(node_root, 'object_num')
    # 一个目标
    node_object_num.text = str(1)

    # 图像尺寸（长宽以及通道数）
    node_size = SubElement(node_root, 'size')
    node_width = SubElement(node_size, 'width')
    node_width.text = '' + width

    node_height = SubElement(node_size, 'height')
    node_height.text = '' + height

    node_depth = SubElement(node_size, 'depth')
    node_depth.text = '3'

    #多个目标是可以加循环
    node_object = SubElement(node_root, 'object')
    node_name = SubElement(node_object, 'name')
    node_name.text = '打火机'
    # 目标是否难以识别（0表示容易识别）
    node_difficult = SubElement(node_object, 'difficult')
    node_difficult.text = '0'

    # bounding-box（包含左下角和右上角xy坐标）
    node_bndbox = SubElement(node_object, 'bndbox')
    node_xmin = SubElement(node_bndbox, 'xmin')
    node_xmin.text = str(xmin_tuple)
    node_ymin = SubElement(node_bndbox, 'ymin')
    node_ymin.text = str(ymin_tuple)
    node_xmax = SubElement(node_bndbox, 'xmax')
    node_xmax.text = str(xmax_tuple)
    node_ymax = SubElement(node_bndbox, 'ymax')
    node_ymax.text = str(ymax_tuple)

    xml = tostring(node_root, pretty_print = True)
    dom = parseString(xml)
    #print xml 打印查看结果
    return dom

In [ ]:
# 处理数据集 把数据集按8:2分成训练和验证集
import os
import random
from PIL import Image # 用于读取图片

filePath = 'work/Fires/train/images'
list_data=os.listdir(filePath)
test_len = (int)(len(list_data) * 0.8)

num = 0
file = open('work/Fires/train/train_list.txt', 'w+')
for i in list_data:
    # 用于区分训练和验证集
    num += 1
    i = i.split(".")

    # 读取图片
    img = Image.open(filePath + '/' +i[0] + '.jpg')
    width = (str)(img.size[0])
    height = (str)(img.size[1])
    # 读取目标左上和右下坐标
    f = open('work/Fires/train/annotations/' + i[0] + '.txt')
    line = f.readline()
    coordinate = line.split()
    xmin_tuple = coordinate[2]
    ymin_tuple = coordinate[3]
    xmax_tuple = coordinate[4]
    ymax_tuple = coordinate[5]
    # 生成XMl文件
    xml = make_xml(xmin_tuple, ymin_tuple, xmax_tuple, ymax_tuple, i[0], width, height)
    f_xml = open('work/Fires/train/annotationxml/' + i[0] + '.xml', 'wb')
    f_xml.write(xml.toprettyxml(indent='\t', encoding='utf-8'))
    f_xml.close()

    if(num < test_len):
        # 生成train_list文件
        file.write('images/' + i[0] +'.jpg annotationxml/' + i[0] +'.xml\n')

num = 0
file = open('work/Fires/train/‪val_list.txt', 'w+')
for i in list_data:
    num += 1
    i = i.split(".")
    if(num > test_len):
        file.write('images/' + i[0] +'.jpg annotationxml/' + i[0] +'.xml\n')

file.close()

In [ ]:
file = open('work/Fires/train/labels.txt', 'w+')
file.write('打火机')
file.close()

# 配置环境

In [ ]:
# 环境变量配置，用于控制是否使用GPU
# '0'使用0号GPU卡进行训练 ''不使用GPU，使用CPU进行训练
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
# 导入paddlx
from paddlex.det import transforms
import paddlex as pdx

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


In [ ]:
# 定义训练和验证时的transforms
train_transforms = transforms.Compose([
    transforms.MixupImage(mixup_epoch=250),
    transforms.RandomDistort(),
    transforms.RandomExpand(),
    transforms.RandomCrop(),
    transforms.Resize(
        target_size=608, interp='RANDOM'),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(),
])

eval_transforms = transforms.Compose([
    transforms.Resize(
        target_size=608, interp='CUBIC'),
    transforms.Normalize(),
])

In [ ]:
# 定义训练和验证所用的数据集
# paddlex.datasets.VOCDetection(
#    data_dir,                    数据集所在的目录路径
#    file_list,                   描述数据集图片文件和对应标注文件的文件路径（文本内每行路径为相对data_dir的相对路径）
#    label_list,                  描述数据集包含的类别信息文件路径
#    transforms=None,             数据集中每个样本的预处理/增强算子
#    num_workers=‘auto’,          数据集中样本在预处理过程中的线程或进程数。默认为’auto’。当设为’auto’时，根据系统的实际CPU核数
#    设置num_workers:如果CPU核数的一半大于8，则num_workers为8，否则为CPU核数的一半
#    buffer_size=100,             数据集中样本在预处理过程中队列的缓存长度，以样本数为单位。默认为100
#    parallel_method='process',   数据集中样本在预处理过程中并行处理的方式，
#    支持’thread’线程和’process’进程两种方式。默认为’process’（Windows和Mac下会强制使用thread，该参数无效）
#    shuffle=False                是否需要对数据集中样本打乱顺序。默认为False


train_dataset = pdx.datasets.VOCDetection(
    data_dir='work/Fires/train',
    file_list='work/Fires/train/train_list.txt',
    label_list='work/Fires/train/labels.txt',
    transforms=train_transforms,
    shuffle=True)
eval_dataset = pdx.datasets.VOCDetection(
    data_dir='work/Fires/train',
    file_list='work/Fires/train/‪val_list.txt',
    label_list='work/Fires/train/labels.txt',
    transforms=eval_transforms)


2021-01-15 19:37:37 [INFO]	Starting to read file list from dataset...
2021-01-15 19:37:38 [INFO]	563 samples in file work/Fires/train/train_list.txt
creating index...
index created!
2021-01-15 19:37:38 [INFO]	Starting to read file list from dataset...
2021-01-15 19:37:38 [INFO]	142 samples in file work/Fires/train/âªval_list.txt
creating index...
index created!


In [ ]:
# 初始化模型，并进行训练
num_classes = len(train_dataset.labels)
model = pdx.det.PPYOLO(num_classes=num_classes)

In [ ]:
model.train(
    num_epochs=270,
    train_dataset=train_dataset,
    train_batch_size=16,
    eval_dataset=eval_dataset,
    learning_rate=0.000125,
    lr_decay_epochs=[210, 240],
    save_dir='output/ppyolo',
    use_vdl=True)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:297: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/detection/ops.py:131
The behavior of expression A * B has been unified with elementwise_mul(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_mul(X, Y, axis=0) instead of A * B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:297: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/detection/loss/yolo_loss.py:155
The behavior of expression A * B has been unified with elementwise_mul(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elem

2021-01-15 19:38:08 [INFO]	Connecting PaddleHub server to get pretrain weights...
[==================================================] 100.00%
Uncompress /home/aistudio/.paddlehub/tmp/tmp5718leu9/ResNet50_vd_ssld_pretrained.tar
[==================================================] 100.00%
2021-01-15 19:38:22 [INFO]	Load pretrain weights from output/ppyolo/pretrain/ResNet50_vd_ssld.
2021-01-15 19:38:22 [INFO]	There are 275 varaibles in output/ppyolo/pretrain/ResNet50_vd_ssld are loaded.
2021-01-15 19:38:39 [INFO]	[TRAIN] Epoch=1/270, Step=2/35, loss=5263.969727, lr=0.0, time_each_step=8.29s, eta=22:2:44
2021-01-15 19:38:41 [INFO]	[TRAIN] Epoch=1/270, Step=4/35, loss=16717.701172, lr=0.0, time_each_step=4.57s, eta=12:9:0
2021-01-15 19:38:42 [INFO]	[TRAIN] Epoch=1/270, Step=6/35, loss=10537.550781, lr=1e-06, time_each_step=3.32s, eta=8:48:47
2021-01-15 19:38:44 [INFO]	[TRAIN] Epoch=1/270, Step=8/35, loss=6464.009277, lr=1e-06, time_each_step=2.67s, eta=7:6:1
2021-01-15 19:38:46 [INFO]	[TRA

100%|██████████| 9/9 [00:08<00:00,  1.11it/s]


2021-01-15 19:57:43 [INFO]	[EVAL] Finished, Epoch=20, bbox_map=0.034195 .
2021-01-15 19:57:51 [INFO]	Model saved in output/ppyolo/best_model.
2021-01-15 19:57:58 [INFO]	Model saved in output/ppyolo/epoch_20.
2021-01-15 19:57:58 [INFO]	Current evaluated best model in eval_dataset is epoch_20, bbox_map=0.03419502660102112
2021-01-15 19:58:07 [INFO]	[TRAIN] Epoch=21/270, Step=2/35, loss=16.147327, lr=8.8e-05, time_each_step=1.71s, eta=3:53:22
2021-01-15 19:58:09 [INFO]	[TRAIN] Epoch=21/270, Step=4/35, loss=15.368773, lr=8.8e-05, time_each_step=1.72s, eta=3:53:19
2021-01-15 19:58:12 [INFO]	[TRAIN] Epoch=21/270, Step=6/35, loss=14.445616, lr=8.8e-05, time_each_step=1.73s, eta=3:53:15
2021-01-15 19:58:15 [INFO]	[TRAIN] Epoch=21/270, Step=8/35, loss=13.599152, lr=8.8e-05, time_each_step=1.7s, eta=3:53:11
2021-01-15 19:58:19 [INFO]	[TRAIN] Epoch=21/270, Step=10/35, loss=15.450215, lr=8.9e-05, time_each_step=1.75s, eta=3:53:9
2021-01-15 19:58:23 [INFO]	[TRAIN] Epoch=21/270, Step=12/35, loss=15.

100%|██████████| 9/9 [00:08<00:00,  1.09it/s]


...

100%|██████████| 9/9 [00:07<00:00,  1.14it/s]


2021-01-15 23:27:44 [INFO]	[EVAL] Finished, Epoch=240, bbox_map=31.124584 .
2021-01-15 23:27:51 [INFO]	Model saved in output/ppyolo/best_model.
2021-01-15 23:27:59 [INFO]	Model saved in output/ppyolo/epoch_240.
2021-01-15 23:27:59 [INFO]	Current evaluated best model in eval_dataset is epoch_240, bbox_map=31.124584381427887
2021-01-15 23:28:07 [INFO]	[TRAIN] Epoch=241/270, Step=2/35, loss=10.4906, lr=1e-06, time_each_step=1.67s, eta=0:28:5
2021-01-15 23:28:09 [INFO]	[TRAIN] Epoch=241/270, Step=4/35, loss=10.970524, lr=1e-06, time_each_step=1.68s, eta=0:28:2
2021-01-15 23:28:12 [INFO]	[TRAIN] Epoch=241/270, Step=6/35, loss=11.227425, lr=1e-06, time_each_step=1.71s, eta=0:28:0
2021-01-15 23:28:15 [INFO]	[TRAIN] Epoch=241/270, Step=8/35, loss=10.096272, lr=1e-06, time_each_step=1.7s, eta=0:27:56
2021-01-15 23:28:18 [INFO]	[TRAIN] Epoch=241/270, Step=10/35, loss=9.843545, lr=1e-06, time_each_step=1.69s, eta=0:27:52
2021-01-15 23:28:20 [INFO]	[TRAIN] Epoch=241/270, Step=12/35, loss=9.891095,

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/reader.py:1255: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn('Your reader has raised an exception!')
Process Process-2068:
Process Process-2072:
Traceback (most recent call last):
Exception in thread Thread-275:
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/reader.py", line 1256, in __thread_main__
    six.reraise(*sys.exc_info())
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/six.py", line 703, in reraise
    raise value
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/

KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/six.py", line 703, in reraise
    raise value
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/datasets/dataset.py", line 162, in _read_into_queue
    queue.put(result)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process Process-2069:
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/datasets/shared_queue/queue.py", line 70, in put
    buff.put(obj)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/mul

In [ ]:
test_jpg = 'work/Fires/test/images/000021401013999.jpg'
model = pdx.load_model('output/ppyolo/best_model')
result = model.predict(test_jpg)
print("Predict Result: ", result[0])

2021-01-15 23:35:56 [INFO]	Model[PPYOLO] loaded.
Predict Result:  {'category_id': 0, 'bbox': [341.90472412109375, 499.57623291015625, 59.13092041015625, 117.0106201171875], 'score': 0.6063533425331116, 'category': '打火机'}


In [30]:

inputStr = "" + (str)(result[0]["score"]) 
print(inputStr)

0.6063533425331116


In [35]:
testFilePath = 'work/Fires/test/images/'
list_data=os.listdir(testFilePath)
file = open('work/result.txt', 'w+')
for i in list_data:
    result = model.predict(testFilePath + i)
    print(i)
    i = i.split(".")
    inputStr = i[0] + " "
    inputStr += (str)(result[0]["score"]) + " "
    inputStr += (str)(result[0]["bbox"][0]) + " "
    inputStr += (str)(result[0]["bbox"][1]) + " "
    inputStr += (str)(result[0]["bbox"][2]) + " "
    inputStr += (str)(result[0]["bbox"][3]) + "\n"
    file.write(inputStr)
file.close()

001373001028515.jpg
001275801027543.jpg
006651001031913.jpg
007468401030557.jpg
007066901030812.jpg
000617801019963.jpg
007132201031488.jpg
007177001031964.jpg
006928501034688.jpg
007886901029830.jpg
001142601026211.jpg
000496701018752.jpg
007704801032921.jpg
007472401030597.jpg
000028901014074.jpg
001243701027222.jpg
006456301027524.jpg
006847401033877.jpg
000853201022317.jpg
007209101032300.jpg
000520001018985.jpg
001061001025395.jpg
006650601031909.jpg
006821301033616.jpg
000409301017878.jpg
007453801030411.jpg
007396701029840.jpg
007461201030485.jpg
007076401030911.jpg
006860201034005.jpg
007515901031032.jpg
006674201032145.jpg
006946101034864.jpg
007832201029283.jpg
000180401015589.jpg
007325601033545.jpg
000132801015113.jpg
001167301026458.jpg
001342201028207.jpg
007432601030199.jpg
000294101016726.jpg
006703201032435.jpg
000201301015798.jpg
006728001032683.jpg
000570001019485.jpg
007198001032185.jpg
001084201025627.jpg
001111701025902.jpg
007886501029826.jpg
000214101015926.jpg
